In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [2]:
DATASET_DIR =r"C:\Users\ishan\OneDrive\Desktop\face recog\newf" # <-- CHANGE THIS TO YOUR ACTUAL PATH

# --- LANDMARK SETUP ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

def extract_metrics(landmarks, shape):
    h, w = shape[:2]

    def denorm(pt):
        return np.array([pt.x * w, pt.y * h])

    # Select key points
    nose = denorm(landmarks[1])
    left_eye = denorm(landmarks[33])
    right_eye = denorm(landmarks[263])
    left_mouth = denorm(landmarks[61])
    right_mouth = denorm(landmarks[291])
    chin = denorm(landmarks[152])

    # Compute distances & ratios
    eye_dist = np.linalg.norm(left_eye - right_eye)
    mouth_width = np.linalg.norm(left_mouth - right_mouth)
    face_height = np.linalg.norm(nose - chin)

    return np.array([
        eye_dist,
        mouth_width,
        face_height,
        eye_dist / mouth_width if mouth_width else 0,
        eye_dist / face_height if face_height else 0
    ])

def extract_all_face_vectors(dataset_dir):
    person_vectors = {}

    for person in os.listdir(dataset_dir):
        person_path = os.path.join(dataset_dir, person)
        if not os.path.isdir(person_path):
            continue
        vectors = []
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            result = face_mesh.process(rgb)
            if result.multi_face_landmarks:
                metrics = extract_metrics(result.multi_face_landmarks[0].landmark, image.shape)
                vectors.append(metrics)
        if vectors:
            person_vectors[person] = vectors
    return person_vectors

In [21]:
import random
def generate_pairs(person_vectors, max_pairs_per_class=500):
    X, y = [], []
    people = list(person_vectors.keys())
    
    # SAME-PERSON
    same_pairs = []
    for person in people:
        vecs = person_vectors[person]
        for i in range(len(vecs)):
            for j in range(i+1, len(vecs)):
                same_pairs.append((vecs[i], vecs[j]))
    random.shuffle(same_pairs)
    same_pairs = same_pairs[:max_pairs_per_class]
    for a, b in same_pairs:
        X.append(np.concatenate([a, b]))
        y.append(1)

    # DIFFERENT-PERSON
    diff_pairs = []
    for i in range(len(people)):
        for j in range(i+1, len(people)):
            for a in person_vectors[people[i]]:
                for b in person_vectors[people[j]]:
                    diff_pairs.append((a, b))
    random.shuffle(diff_pairs)
    diff_pairs = diff_pairs[:max_pairs_per_class]
    for a, b in diff_pairs:
        X.append(np.concatenate([a, b]))
        y.append(0)

    return np.array(X), np.array(y)


In [22]:
person_vectors = extract_all_face_vectors(DATASET_DIR)
X, y = generate_pairs(person_vectors)

In [23]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class FaceMLP(nn.Module):
    def __init__(self, input_size=10):
        super(FaceMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [26]:
def prepare_data(X, y, batch_size=32, val_ratio=0.2):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    dataset = TensorDataset(X_tensor, y_tensor)
    val_size = int(len(dataset) * val_ratio)
    train_size = len(dataset) - val_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)

    return train_loader, val_loader


In [27]:
def train_mlp(X, y, epochs=10, lr=0.001):
    train_loader, val_loader = prepare_data(X, y)

    model = FaceMLP(input_size=X.shape[1]).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                predicted = (preds > 0.5).float()
                correct += (predicted == yb).sum().item()
                total += yb.size(0)

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f} - Val Accuracy: {accuracy:.4f}")

    return model


In [31]:
model = train_mlp(X, y)

Epoch 1/10 - Loss: 19.7031 - Val Accuracy: 0.5400
Epoch 2/10 - Loss: 17.9579 - Val Accuracy: 0.5250
Epoch 3/10 - Loss: 18.0786 - Val Accuracy: 0.4900
Epoch 4/10 - Loss: 17.4013 - Val Accuracy: 0.5150
Epoch 5/10 - Loss: 17.5925 - Val Accuracy: 0.4900
Epoch 6/10 - Loss: 17.7954 - Val Accuracy: 0.5100
Epoch 7/10 - Loss: 17.3125 - Val Accuracy: 0.4800
Epoch 8/10 - Loss: 17.2738 - Val Accuracy: 0.5000
Epoch 9/10 - Loss: 17.4300 - Val Accuracy: 0.5200
Epoch 10/10 - Loss: 17.6084 - Val Accuracy: 0.5350


In [32]:
print(X.shape)

(1000, 10)


In [29]:
print("Using device:", device)

Using device: cuda


In [30]:
from collections import Counter
print(Counter(y)) 

Counter({1: 500, 0: 500})
